In [1]:
import sqlite3
import pandas as pd
import sqlalchemy
import numpy

In [3]:
#Подключаем SQL базу данных
conn_db = sqlite3.connect("bd/main_bd.sqlite", timeout=10)
c = conn_db.cursor()
c.execute("PRAGMA foreign_keys = ON;")

In [6]:
#ФИЛЬМЫ
movies = pd.read_csv(("bd/root_dataset/movies.csv"), sep=';')
print(movies)
##Отделяем название от года выпуска
#titlename = [i[:-7] for i in movies["title"]]
#titleyear = [i[-5:-1] for i in movies["title"]]
#movies["title"].update(titlename)
#movies["year"] = titleyear

                                           Title  title_year     budget  \
0                                     La La Land        2016   30000000   
1                                       Zootopia        2016  150000000   
2                                           Lion        2016   12000000   
3                                        Arrival        2016   47000000   
4                          Manchester by the Sea        2016    9000000   
..                                           ...         ...        ...   
95                                      Whiplash        2014    3300000   
96                               Before Midnight        2013    3000000   
97    Star Wars: Episode VII - The Force Awakens        2015  245000000   
98  Harry Potter and the Deathly Hallows: Part I        2010  150000000   
99                       Tucker and Dale vs Evil        2010    5000000   

        Gross              actor_1_name        actor_2_name  \
0   151101803              Ryan Gosl

In [37]:

#Реестр всех жанров
GENRES_REG = None
for i in movies["genres"]:
    if " " not in i:
        GENRES_REG = set(i.split('|'))
        break
#Заносим в множество реестр всех жанров все жанры по всем фильмам, отсутствие жанра помечаем пустой строкой, в остальных строках заменяем "|" на ","
for i in range(len(movies["genres"])):
    if " " in movies["genres"].loc[i]:
        movies["genres"].loc[i] = '' #заменяем список жанров каждого фильма, у которого они не указаны на пустую строку
        pass
    else:
        genrs = set(movies["genres"].loc[i].split("|")) #Заменяем значения исходной базы данных
        movies["genres"].loc[i] = movies["genres"].loc[i].replace("|", ",")
        GENRES_REG.update(genrs) #Обновляем реестр
#reestr = open("bd/genresReg.txt", "w")
#reestr.write((",").join(GENRES_REG))
#reestr.close()

C:\Users\Пользователь\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [16]:
#Создаем и заполняем таблицу movies
c.execute(
    """
    DROP TABLE IF EXISTS movies;
    """
    )
c.execute(
    """
    CREATE TABLE IF NOT EXISTS movies(
        movieId INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        genres TEXT NOT NULL,
        year INT NOT NULL
        );
    """
    )
movies.to_sql('movies', conn_db, if_exists='append', index=False)
conn_db.commit()

In [43]:
#ОЦЕНКИ
ratings = pd.read_csv(("bd/root_dataset/ratings.csv"))

In [38]:
#ПОЛЬЗОВАТЕЛИ 
#Создаем SQL таблицу пользователей
c.execute(
    """
    DROP TABLE IF EXISTS users;
    """
    )
c.execute(
    """
    CREATE TABLE IF NOT EXISTS users(
        userId INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT NULL UNIQUE,
        password TEXT NULL
        );
    """
    )
#ЗАГРУЖАЕМ ВСЕХ ИЗНАЧАЛЬНЫХ ФИКТИВНЫХ ПОЛЬЗОВАТЕЛЕЙ
c.execute(
    f"""
    INSERT INTO users(userId) VALUES {','.join(f'({i})' for i in ratings['userId'].unique())};"""
    )
conn_db.commit()

In [44]:
#ОЦЕНКИ ПОЛЬЗОВАТЕЛЕЙ
#СОЗДАЕМ SQL ТАБЛИЦУ
c.execute(
    """
    DROP TABLE IF EXISTS ratings;
    """
    )
c.execute(
    """
    CREATE TABLE IF NOT EXISTS ratings(
        userId INT NOT NULL,
        movieId INT NOT NULL,
        rating INT NULL,
        timestamp TEXT NOT NULL,
        PRIMARY KEY(userId, movieId),
        FOREIGN KEY(userId) REFERENCES users(userId) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY(movieId) REFERENCES movies(movieId) ON DELETE CASCADE ON UPDATE CASCADE
        );
    """
    )
ratings.to_sql('ratings', conn_db, if_exists='append', index=False)
conn_db.commit()

In [40]:
#ТЕГИ
tags = pd.read_csv(("bd/root_dataset/tags.csv"))
tags['tag'] = tags['tag'].apply(lambda x: x.lower().strip())

In [41]:
#ТЕГИ ФИЛЬМОВ ОТ ПОЛЬЗОВАТЕЛЕЙ
#СОЗДАЕМ SQL ТАБЛИЦУ
c.execute(
    """
    DROP TABLE IF EXISTS tags;
    """
    )
c.execute(
    """
    CREATE TABLE IF NOT EXISTS tags(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        userId INT NOT NULL,
        movieId INT NOT NULL,
        tag TEXT NOT NULL,
        timestamp TEXT NOT NULL,
        FOREIGN KEY(userId) REFERENCES users(userId) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY(movieId) REFERENCES movies(movieId) ON DELETE CASCADE ON UPDATE CASCADE,
        UNIQUE(userId, movieId, tag)
        );
    """
    )
tags.to_sql('tags', conn_db, if_exists='append', index=True, index_label='id')
conn_db.commit()

In [42]:
#ТАБЛИЦА С ИЗБРАННЫМИ ФИЛЬМАМИ ПОЛЬЗОВАТЕЛЕЙ
c.execute(
    """
    DROP TABLE IF EXISTS users_favorites;
    """
    )
c.execute(
    """
    CREATE TABLE IF NOT EXISTS users_favorites(
        userId INT NOT NULL,
        movieId INT NOT NULL,
        timestamp TEXT NOT NULL,
        PRIMARY KEY(userId,movieId),
        FOREIGN KEY(userId) REFERENCES users(userId) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY(movieId) REFERENCES movies(movieId) ON DELETE CASCADE ON UPDATE CASCADE
        );
    """
    )
conn_db.commit()

In [26]:
#ОТОБРАЖЕНИЕ ФИЛЬМОВ И ЕГО СРЕДНИХ ОЦЕНОК
c.execute( 
    """
    DROP VIEW IF EXISTS movies_ratings;
    """
)
c.execute(
    """
    CREATE VIEW movies_ratings AS 
    SELECT movies.movieId, AVG(r.rating) as avg_score, COUNT(r.rating) as count_scores 
    FROM movies
    LEFT JOIN ratings r ON r.movieId = movies.movieId
    GROUP BY movies.movieId
    ORDER BY 1
    """
)
conn_db.commit()

In [ ]:
#ОТОБРАЖЕНИЕ ФИЛЬМОВ, ЕГО ТЕГОВ И КОЛИЧЕСТВО ОТМЕТОК КАЖДОГО ПРИСОВОЕНОГО ТЕГА
c.execute(
    """
    DROP VIEW IF EXISTS movies_tags;
    """
)
c.execute(
    """
    CREATE VIEW movies_tags AS 
    SELECT movieId, tag, COUNT(tag) as count 
    FROM tags
    GROUP BY movieId, tag; 
    """
)
conn_db.commit()

In [ ]:
#ТРИГГЕР НА users НА INSERT
c.execute(
    """
    CREATE TRIGGER unique_username BEFORE INSERT ON users
    WHEN EXISTS (SELECT * FROM users WHERE LOWER(username) = LOWER(NEW.username))
    BEGIN
        SELECT RAISE(ABORT, 'Unique username');
    END;
    """
)
conn_db.commit()

In [8]:
a = ['NULL']
#func = lambda x, y: f'{y} LIKE \'%' + f'%\' AND {y} LIKE \'%'.join(x) + '\''
#print(func(a, 'genres'))
#print(f"({','.join(a)})")
c.execute(
    f"""
    INSERT INTO ratings VALUES (611, 1, 5, 123123)
    """
)
conn_db.commit()
print(c.fetchall())

ProgrammingError: Cannot operate on a closed database.

In [7]:
conn_db.commit()
conn_db.close()

In [ ]:
#Создаем дата фрейм с ид фильмов и соответствующими им средними оценками и количеством оценок, взятых из ratings
"""
movies_ratings = pd.DataFrame({
    'movieId' : movies['movieId'],
    'avg_score' : [0] * len(movies['movieId']),
    'count_scores' : [0] * len(movies['movieId'])
})
for i in ratings['movieId'].unique():
    scores = ratings['rating'].loc[ratings['movieId'] == i]
    c_scores = len(scores)
    a_score = scores.sum() / c_scores
    movies_ratings['avg_score'].loc[movies_ratings['movieId'] == i] = a_score
    movies_ratings['count_scores'].loc[movies_ratings['movieId'] == i] = c_scores
print(movies_ratings)
"""

In [ ]:
"""
movies_tags = pd.DataFrame({
    'movieId' : [], #tags['movieId'].unique(),
    'tag' : [],
    'count': [] #[ [tags['tag'].loc[tags['movieId'] == i]] for i in (tags.sort_values('movieId'))['movieId'].unique()]
})
for i in range(len(tags)):
    movieId = tags.loc[i]['movieId']
    tag = tags.loc[i]['tag']
    if movies_tags.loc[(movies_tags['movieId'] == movieId) & (movies_tags['tag'] == tag)].empty:
        movies_tags = movies_tags.append({'movieId' : movieId, 'tag' : tag, 'count' : 1}, ignore_index=True, verify_integrity=True).astype({'movieId' : "int64", "count" : "int64"})
    else:
        movies_tags['count'].loc[(movies_tags['movieId'] == movieId) & (movies_tags['tag'] == tag)] += 1
print(movies_tags.sort_values('movieId'))
"""